In [24]:
from geopy.distance import geodesic
import requests

def get_nearest_station(lat, lon, radius_km=100):
    """
    Devuelve la estación más cercana de OpenAQ dentro del radio dado.
    Si no hay estaciones en el radio, devuelve None.
    """
    url = "https://api.openaq.org/v3/locations"
    params = {
        "coordinates": f"{lat},{lon}",
        "radius": radius_km * 1000,  # en metros
        "limit": 100
    }

    r = requests.get(url, headers=headers, params=params)
    if r.status_code != 200:
        raise Exception(f"Error {r.status_code}: {r.text}")

    results = r.json().get("results", [])
    if not results:
        return None

    # calcular distancia geodésica y elegir la más cercana
    for s in results:
        s["distance_km"] = geodesic(
            (lat, lon),
            (s["coordinates"]["latitude"], s["coordinates"]["longitude"])
        ).km

    nearest = min(results, key=lambda x: x["distance_km"])
    return nearest

def has_station_nearby(lat, lon, max_km=25):
    params = {"coordinates": f"{lat},{lon}", "radius": max_km*1000}
    r = requests.get("https://api.openaq.org/v3/locations", headers=headers, params=params)
    return len(r.json().get("results", [])) > 0


In [7]:
def get_station_data(station):
    """
    Extrae de un objeto de estación (ya devuelto por get_nearest_station)
    los valores más recientes de cada contaminante.
    """
    sensors = station.get("sensors", [])
    data = []
    for s in sensors:
        param = s["parameter"]
        data.append({
            "station_id": station["id"],
            "station_name": station["name"],
            "parameter": param["name"],
            "display": param["displayName"],
            "unit": param["units"],
            "sensor_name": s["name"],
            "sensor_id": s["id"],
            "datetime_last": station.get("datetimeLast", {}).get("utc"),
            "country": station["country"]["name"],
            "provider": station["provider"]["name"],
            "distance_km": round(station["distance_km"], 3),
        })
    return data



In [19]:
import requests
from datetime import datetime, timedelta, timezone

def get_station_measurements(location_id, hours=24):
    """
    Devuelve los valores recientes de TODOS los sensores de una estación.
    """
    import requests
    from datetime import datetime, timedelta, timezone

    url = "https://api.openaq.org/v3/measurements"
    date_to = datetime.now(timezone.utc)
    date_from = date_to - timedelta(hours=hours)

    params = {
        "location_id": location_id,
        "date_from": date_from.isoformat(timespec="seconds"),
        "date_to": date_to.isoformat(timespec="seconds"),
        "limit": 200,
        "sort": "desc"
    }

    r = requests.get(url, headers=headers, params=params)
    if r.status_code != 200:
        raise Exception(f"Error {r.status_code}: {r.text}")

    data = r.json()["results"]

    # Tomar el último valor por parámetro
    latest = {}
    for m in data:
        param = m["parameter"]["name"]
        if param not in latest:
            latest[param] = {
                "display": m["parameter"]["displayName"],
                "value": m["value"],
                "unit": m["unit"],
                "utc": m["date"]["utc"]
            }

    return latest

In [22]:
from dotenv import load_dotenv
import os
load_dotenv()

API_KEY = api_key = os.getenv("API_KEY")

headers = {"X-API-Key": API_KEY}

station = get_nearest_station(19.4326, -99.1332, radius_km=25)

print(station)
for s in station["sensors"]:
    print(f" - {s['parameter']['displayName']} ({s['parameter']['name']}) [{s['parameter']['units']}]")

latest_data = get_station_data(station)
for d in latest_data:
    print(f"{d['display']}: {d['sensor_name']} ({d['unit']})  Último dato: {d['datetime_last']}")


{'id': 10748, 'name': 'Merced', 'locality': 'Venustiano Carranza', 'timezone': 'America/Mexico_City', 'country': {'id': 157, 'code': 'MX', 'name': 'Mexico'}, 'owner': {'id': 4, 'name': 'Unknown Governmental Organization'}, 'provider': {'id': 223, 'name': 'Sinaica Mexico'}, 'isMobile': False, 'isMonitor': True, 'instruments': [{'id': 2, 'name': 'Government Monitor'}], 'sensors': [{'id': 35864, 'name': 'co ppm', 'parameter': {'id': 8, 'name': 'co', 'units': 'ppm', 'displayName': 'CO'}}, {'id': 4275660, 'name': 'no ppm', 'parameter': {'id': 35, 'name': 'no', 'units': 'ppm', 'displayName': 'NO'}}, {'id': 35861, 'name': 'no2 ppm', 'parameter': {'id': 7, 'name': 'no2', 'units': 'ppm', 'displayName': 'NO₂'}}, {'id': 4275914, 'name': 'nox ppm', 'parameter': {'id': 19840, 'name': 'nox', 'units': 'ppm', 'displayName': 'NOx'}}, {'id': 35860, 'name': 'o3 ppm', 'parameter': {'id': 10, 'name': 'o3', 'units': 'ppm', 'displayName': 'O₃'}}, {'id': 35857, 'name': 'pm10 µg/m³', 'parameter': {'id': 1, 'na

In [23]:
print(get_station_measurements(station["id"]))


Exception: Error 404: {"detail":"Not Found"}